In [ ]:
%matplotlib notebook
from risk_dashboard import NpvCube
from ipywidgets import SelectMultiple
from IPython.display import display

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib import cm
from enum import Enum
from matplotlib.font_manager import FontProperties

csv_file = 'cube.csv'
npv = NpvCube(csv_file)

current_trades = [0]
def set_current_trades(self, trades):
    current_trades = trades['new']
trades_selector = SelectMultiple(options=npv.tradedict, description='TradeIDs:')
trades_selector.observe(set_current_trades, names='value')
display(trades_selector)

In [ ]:
class ExposureTypes(Enum):
    npv = 'NPV'
    npvp = 'NPV+'
    npvm = 'NPV-'
    
current_exposure=ExposureTypes.npv

npv.calc_density_data(current_trades, current_exposure)
fig_surface = plt.figure()
ax_surface =fig_surface.add_subplot(111, projection='3d')

# configure x-axis to be labeld with dates
date_step = 5
ax_surface.set_xticks(np.arange(0, len(npv.dates), date_step))
ax_surface.set_xticklabels(npv.dates[0::date_step])
ax_surface.set_xlabel('Dates')
# set thousands separator for y-axis
ax_surface.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(
    lambda x, p: format(int(x), ',')))

ax_surface.set_ylabel(current_exposure.value)

# estimate the probability density for each date
# create X and Y data for plotting
X, Y = np.meshgrid(np.arange(len(npv.dates)), npv.dist_space)
# ax.plot_surface(X, Y, values.T, cmap=cm.jet)
ax_surface.plot_trisurf(
    X.flatten(),
    Y.flatten(),
    npv.density_values.T.flatten(),
    cmap=cm.jet,
    linewidth=0)
plt.show()

In [ ]:
npv.calc_exposure_statistics(current_trades)
fig_statistics = plt.figure()
ax_statistics = fig_statistics.add_subplot(111)
ax_statistics.plot(range(len(npv.dates)), npv.exposure_statistics['npv'], label='NPV')
ax_statistics.plot(range(len(npv.dates)), npv.exposure_statistics['npv+'], label='NPV+')
ax_statistics.plot(range(len(npv.dates)), npv.exposure_statistics['npv-'], label='NPV-')
ax_statistics.plot(range(len(npv.dates)), npv.exposure_statistics['perc'], label='perc')
ax_statistics.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
date_step = 2
ax_statistics.set_xticks(np.arange(0, len(npv.dates), date_step))
ax_statistics.set_xticklabels(npv.dates[0::date_step], rotation=90)
fontP = FontProperties() # set legend outside plot
fontP.set_size('small')
ax_statistics.legend(loc='upper right', shadow=True, prop=fontP)
plt.show()